In [67]:
import json
import os
import re
from datetime import datetime, timedelta
from urllib.parse import urlparse
import time
import yaml

import pandas as pd 

import sqlite3

import pyperclip

import bs4
from bs4 import BeautifulSoup
import requests

import dotenv

import IPython
from IPython.display import HTML, Markdown, display

from atproto import Client

import PIL
from PIL import Image

# disable 'show more outputs' nonsense
from IPython.core.interactiveshell import InteractiveShell

# Always display all outputs in a cell
InteractiveShell.ast_node_interactivity = "all"


In [68]:
print(f"requests        {requests.__version__}")
print(f"BeautifulSoup   {bs4.__version__}")


requests        2.32.3
BeautifulSoup   4.13.4


In [69]:
dotenv.load_dotenv()


True

# Load posts from BlueSky and format for Substack or a blog post
for now I share the interesting stuff on bluesky and then use this code to grab latest BlueSky 'tweets' and format a [Substack post](https://skynetandchill.com)


In [70]:
client = Client(base_url='https://bsky.social')
client.login(os.environ['BSKY_USERNAME'], os.environ['BSKY_SECRET'])

mydid = {"did":"did:plc:qomkdnxrqw3gkbytdxea5z65"}

data = client.get_author_feed(
    actor=mydid['did'],
    filter='posts_and_author_threads',
    limit=80,
)


ProfileViewDetailed(did='did:plc:qomkdnxrqw3gkbytdxea5z65', handle='skynetandchill.com', associated=ProfileAssociated(chat=ProfileAssociatedChat(allow_incoming='following', py_type='app.bsky.actor.defs#profileAssociatedChat'), feedgens=0, labeler=False, lists=0, starter_packs=0, py_type='app.bsky.actor.defs#profileAssociated'), avatar='https://cdn.bsky.app/img/avatar/plain/did:plc:qomkdnxrqw3gkbytdxea5z65/bafkreihi55ddvmohg6dztiwlkwdbts5fd76iaenef2sriacvj7ibtezagi@jpeg', banner='https://cdn.bsky.app/img/banner/plain/did:plc:qomkdnxrqw3gkbytdxea5z65/bafkreigu723bmkba3mff4krhcrrf7b5zw2svixwbvtj54v5ho64o2xpmzu@jpeg', created_at='2024-02-12T00:15:55.099Z', description='Daily artificial intelligence news, lovingly curated by man and machine. https://www.skynetandchill.com\n\nNeo-Luddite AI maven. On a long enough timeline, p(doom) for everything goes to 1.', display_name='SkynetAndChill.com', followers_count=348, follows_count=348, indexed_at='2024-11-21T15:40:08.470Z', joined_via_starter_p

In [71]:
def remove_urls(text):
    # Regular expression to match URLs
    url_pattern = r'https?://\S+|www\.\S+'
    # Substitute found URLs with an empty string
    clean_text = re.sub(url_pattern, '', text)
    return clean_text


In [72]:
def rawfetchurl(url, timeout=60):
    """get url using requests with specified timeout. return response object, status, content-type"""
    try:
        response = requests.get(url, timeout=timeout)
    except httplib.BadStatusLine:
        log("Bad response (?) fetching url %s " % url)
        response = None
    except requests.Timeout:
        log("Timeout fetching url %s " % url)
        response = None
    except requests.ConnectionError as e:
        log("Connection error (%s) fetching url %s " % (str(e), url))
        response = None
    except requests.TooManyRedirects:
        log("Too many redirects fetching url %s " % url)
        response = None
    except requests.exceptions.MissingSchema:
        log("Missing schema url %s " % url)
        response = None
    except requests.exceptions.InvalidSchema:
        log("Invalid schema url %s " % url)
        response = None
    except requests.exceptions.InvalidURL as e:
        log("Invalid url %s, %s" % (url, str(e)))
        response = None
    except ValueError as e:
        # don't log url because possibly malformed url
        log("ValueError, url ?: ? ")
        response = None
    except httplib.IncompleteRead as e:
        log("IncompleteRead, url %s: %s " % (url, str(e)))
        response = None
    except urllib3.exceptions.SSLError as e:
        log("SSLError, url %s: %s " % (url, str(e)))
        response = None
    except requests.exceptions.ContentDecodingError as e:
        log("SSLError, url %s: %s " % (url, str(e)))
        response = None
    except requests.exceptions.ChunkedEncodingError as e:
        log("ChunkedEncodingError, url %s: %s " % (url, str(e)))
        response = None
    except UnicodeEncodeError as e:
        log("UnicodeEncodeError, url %s: %s " % (url, str(e)))
        response = None
    except OpenSSL.SSL.SysCallError as e:
        log("OpenSSL.SSL.SysCallError, url %s: %s " % (url, str(e)))
        response = -1
    except OpenSSL.SSL.ZeroReturnError as e:
        log("OpenSSL.SSL.ZeroReturnError, url %s: %s " % (url, str(e)))
        response = -1

    # except requests.packages.urllib3.exceptions.DecodeError as e:
    #     utilLog("DecodeError, url %s: %s " % (url, str(e)))
    #     response = None

    return response



In [73]:
def resize_and_crop(input_image_path, output_image_path, desired_height=240):
    # Load the image
    with Image.open(input_image_path) as img:
        img = img.convert('RGB')

        # Calculate the new width maintaining the aspect ratio
        aspect_ratio = img.width / img.height
        new_width = int(desired_height * aspect_ratio)

        # Resize the image
        resized_img = img.resize((new_width, desired_height))

        # Save the resized image
        resized_img.save(output_image_path)


In [74]:
# attempt to remove traiing inline URLs

def truncate_last_occurrence(text: str) -> str:
    # Find trailing occurrence of a space followed by any sequence of characters followed by 3 periods
    pattern = r'\s+\S+\.{3}$'
    return re.sub(pattern, '', text)

# Example text for testing

example_text = """Elon Musk says we'll run out of power capacity to run all the AI chips in 2025
newatlas.com/technology/e..."""

# Truncate the last occurrence
print(truncate_last_occurrence(example_text))


Elon Musk says we'll run out of power capacity to run all the AI chips in 2025


In [75]:
def get_og_tags(url):
    """get a dict of Open Graph og: tags such as title in the HEAD of a URL"""
    retdict = {}
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, "html.parser")
            head = soup.head
            if head:
                og_tags = head.find_all(
                    property=lambda prop: prop and prop.startswith("og:")
                )
                for tag in og_tags:
                    if "content" in tag.attrs:
                        retdict[tag["property"]] = tag["content"]

                page_title = ""
                title_tag = soup.find("title")
                if title_tag:
                    page_title = title_tag.text
                    if page_title:
                        retdict["title"] = page_title
        return retdict
    except requests.RequestException as e:
        log(f"Error fetching {url}: {e}")
    return retdict


url = "https://druce.ai"
get_og_tags(url)

{'og:site_name': 'Druce.ai',
 'og:title': 'Druce.ai',
 'og:type': 'website',
 'og:description': "Druce's Blog on Machine Learning, Tech, Markets and Economics",
 'og:url': 'https://druce.ai/',
 'title': 'Druce.ai'}

In [76]:
def delete_files(outputdir):

    # Iterate over all files in the directory
    for filename in os.listdir(outputdir):
        if filename.startswith('.'):
            continue
        file_path = os.path.join(outputdir, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.remove(file_path)  # Remove the file
            elif os.path.isdir(file_path):
                # If you want to remove subdirectories as well, use os.rmdir() here
                pass
        except Exception as e:
            log(f'Failed to delete {file_path}. Reason: {e}')
            

In [77]:
# import sqlite3
# import tldextract

# # Path to your SQLite database
# db_path = "articles.db"

# # Connect to the database
# conn = sqlite3.connect(db_path)
# cur = conn.cursor()

# # Fetch all rows (id, hostname)
# cur.execute("SELECT id, hostname FROM sites")
# rows = cur.fetchall()

# for row in rows:
#     site_id, hostname = row
#     # Extract the registered domain (e.g., 'bbc.co.uk', '9to5mac.com')
#     extracted = tldextract.extract(hostname)
#     if extracted.domain and extracted.suffix:
#         registered_domain = f"{extracted.domain}.{extracted.suffix}"
#         # Update the domain column
#         print(f"{hostname} -> {registered_domain}")
#         cur.execute(
#             "UPDATE sites SET domain = ? WHERE id = ?",
#             (registered_domain, site_id))
#     else:
#         registered_domain = None  # fallback if parsing fails

# # Commit changes and close
# conn.commit()
# conn.close()

In [78]:
imgdir = 'download_images'  # for images
delete_files(imgdir)

conn = sqlite3.connect('articles.db')
query = "select * from sites"
sites_df = pd.read_sql_query(query, conn)
sites_dict = {row.hostname: row.site_name for row in sites_df.itertuples()}
conn.close()


In [79]:
with open("most_recent_bsky.md", 'r') as file:
    prev_start = file.readline().strip()    
prev_start

'Large Language Models Often Know When They Are Being Evaluated'

In [80]:
def parse_feed(prev_start=""):
    display_dict = {}
    image_dict = {}
    for i, post in enumerate(data.feed):
        
        # check date
        date_str = data.feed[0].post.record.created_at
        dt = datetime.fromisoformat(date_str.rstrip('Z'))
        # break if more than 30 hours ago
        if dt  < datetime.now() - timedelta(hours=30):
            break

        # get post_str, post_url
        post_str = post.post.record.text.strip()
        post_str = truncate_last_occurrence(post_str)
        post_str = post_str.replace("$", "\\\$")  # so Markdown doesn't interpret $ as latex escape
        
        if i==0:
            with open("most_recent_bsky.md", 'w') as file:
                file.write(post_str)
                
        # stop if we are up to previous
        if post_str == prev_start:
            print(f"matched previous value at i={i}")
            break

        try:
            post_url = post.post.record.embed.external.uri.rstrip()
        except:
            post_url = ""

        if post_url:
            print(post_url) # to know if it's blocked
            # get site name , image using og tags
            try:
                tag_dict = get_og_tags(post_url)
            except:
                tag_dict = {}

            site_name = tag_dict.get('og:site_name')
            img_url = tag_dict.get('og:image')

            # get site_name from stored values as backup
            if not(site_name):
                parsed_url = urlparse(post_url)
                hostname = parsed_url.hostname
                site_name = sites_dict.get(hostname, hostname)

            md_str = f"[{post_str}]({post_url})  - {site_name}"

            display_dict[i]=md_str

            # write local copy of image
            if img_url:
                try:
                    r = rawfetchurl(img_url)
                    content_type = r.headers['Content-Type']
                    content_type = content_type[content_type.find('/')+1:]
                    impath = f"{imgdir}/source{i}.{content_type}"
                    with open(impath, 'wb') as file:
                        file.write(r.content)
                    output_path = f'{imgdir}/Image{i}.jpg'
                    resize_and_crop(impath, output_path)
                    image_dict[i] = f"![image]({output_path})\n\n"
                except Exception as e:
                    print(e)

        else: # no url
            md_str = f"\n{post_str}" + "\n\n"
            display_dict[i]=md_str
            
    return display_dict, image_dict

display_dict, image_dict = parse_feed(prev_start)
len(display_dict)


https://www.ft.com/content/55bc5876-254a-4daa-86f8-2cd0d939a866
https://www.nytimes.com/2025/06/16/magazine/using-ai-hard-fork.html
https://www.scmp.com/tech/tech-trends/article/3314467/how-chinese-ai-start-helps-nike-gap-urban-revivo-predict-fashion-trends
https://victoramartinez.com/posts/why-claudes-comment-paper-is-a-poor-rebuttal/
https://economictimes.indiatimes.com/magazines/panache/neuralink-babies-scale-ais-alexandr-wang-says-he-is-waiting-for-elon-musks-brain-chips-before-having-kids/articleshow/121867951.cms?utm_source=contentofinterest&utm_medium=text&utm_campaign=cppst
https://newrepublic.com/article/196763/ai-industry-trump-defense-department
https://cointelegraph.com/news/ai-race-between-us-china-cold-war-andreessen
https://thenewstack.io/ready-or-not-agentic-ai-is-disrupting-corporate-landscapes/
https://www.businessinsider.com/ai-coding-tools-buy-versus-build-software-saas-netlify-bolt-2025-6
https://simonwillison.net/2025/Jun/16/the-lethal-trifecta/
https://blog.singl

36

In [81]:
output_str = ""
for i in sorted(display_dict.keys()):
    md_str = f"{i}. "
    md_str += display_dict.get(i)
    if image_dict.get(i):
        md_str += image_dict.get(i)
    md_str += "\n\n---\n\n"
    output_str += md_str

output_str += f"Follow the latest AI headlines via [SkynetAndChill.com on Bluesky](https://bsky.app/profile/skynetandchill.com)"

display(Markdown(output_str))


0. [Economic miracles stem from discovery of deeper insights, not repeating tasks at greater speed. Oxford author of "How Progress Ends: Technology, Innovation, and the Fate of Nations" (sometimes creativity is a search process tho?)](https://www.ft.com/content/55bc5876-254a-4daa-86f8-2cd0d939a866)  - Financial Times![image](download_images/Image0.jpg)



---

1. [Everyone Is Using A.I. for Everything. Is That Bad? So what if we have armies of interns who act like they are high on ketamine?](https://www.nytimes.com/2025/06/16/magazine/using-ai-hard-fork.html)  - The New York Times

---

2. [Chinese start-up applies AI to help Nike, Gap, Urban Revivo predict fashion trends](https://www.scmp.com/tech/tech-trends/article/3314467/how-chinese-ai-start-helps-nike-gap-urban-revivo-predict-fashion-trends)  - South China Morning Post![image](download_images/Image2.jpg)



---

3. [Why the rebuttal to Apple's paper on limits of reasoning models misses the mark.](https://victoramartinez.com/posts/why-claudes-comment-paper-is-a-poor-rebuttal/)  - Victor A Martinez

---

4. [Alexandr Wang waiting for brain implants to have kids: “When we get Neuralink and we get these other technologies, kids who are born with them are gonna learn how to use them in like crazy, crazy ways”](https://economictimes.indiatimes.com/magazines/panache/neuralink-babies-scale-ais-alexandr-wang-says-he-is-waiting-for-elon-musks-brain-chips-before-having-kids/articleshow/121867951.cms?utm_source=contentofinterest&utm_medium=text&utm_campaign=cppst)  - The Economic Times![image](download_images/Image4.jpg)



---

5. [The AI Industry Is Ready to Get Rich off Trump’s Defense Department](https://newrepublic.com/article/196763/ai-industry-trump-defense-department)  - The New Republic![image](download_images/Image5.jpg)



---

6. 




---

7. [AI is the control layer for everything, and race between US and China resembles Cold War.](https://cointelegraph.com/news/ai-race-between-us-china-cold-war-andreessen)  - Cointelegraph

---

8. [Ready or Not, Agentic AI Is Disrupting Corporate Landscapes](https://thenewstack.io/ready-or-not-agentic-ai-is-disrupting-corporate-landscapes/)  - The New Stack![image](download_images/Image8.jpg)



---

9. [AI coding tools upend the 'buy versus build' software equation and threaten the SaaS business model. Sometimes you buy a big SaaS to do 1 or 2 things in your workflow that you could just vibe-code.](https://www.businessinsider.com/ai-coding-tools-buy-versus-build-software-saas-netlify-bolt-2025-6)  - Business Insider![image](download_images/Image9.jpg)



---

10. [the lethal trifecta - 
- access to internal data
- access to external tools
- untrusted prompts / outputs (and any amount of guardrails and sanitizing will eventually be defeated)](https://simonwillison.net/2025/Jun/16/the-lethal-trifecta/)  - Simon Willison’s Weblog![image](download_images/Image10.jpg)



---

11. [Coding agents have crossed a chasm](https://blog.singleton.io/posts/2025-06-14-coding-agents-cross-a-chasm/)  - Singleton Blog

---

12. [Your Brain on ChatGPT: EEG Shows Accumulation of Cognitive Debt when Using an AI Assistant for Essay Writing Task](https://arxiv.org/abs/2506.08872)  - arXiv.org

---

13. 
Elon Musk working on fixing Grok's 'leftist indoctrination', ie lobotomizing it to rant about white genocide.



---

14. [Why Waymo cars became sitting ducks during the L.A. protests.](https://www.nbcnews.com/tech/innovation/waymo-cars-set-fire-sitting-ducks-la-protests-rcna212426)  - NBC News![image](download_images/Image14.jpg)



---

15. [London is a big test for robotaxis. It ain't Phoenix, might not work without people messing with them, cautious taxis gridlocking narrow chaotic streets, needing remote operator intervention and geo-fencing out of certain areas or redesigned roads.](https://www.businessinsider.com/i-took-chaotic-robotaxi-ride-through-london-impressive-one-question-2025-6)  - Business Insider![image](download_images/Image15.jpg)



---

16. 
I was reading this waymo blog post, and remembered that waymo famously drives about 1000x as much in simulation as in the real world. Lines up pretty well with the predictions i made in my "scaling robot learning to Robot GPT" blog post:



---

17. [How AI can help make cities work better for residents. Big opportunities to improve services, and screw up massively.](https://www.technologyreview.com/2025/06/16/1118210/ai-city-government-residents-constituents/)  - MIT Technology Review![image](download_images/Image17.jpg)



---

18. [“If history, per the adage, is written by the winners, then it’s not premature to wonder how the winners of the A.I. race might soon shape the stories that historians tell about the past.”](https://www.nytimes.com/2025/06/16/magazine/ai-history-historians-scholarship.html?unlocked_article_code=1.PU8.EhBm.UkxDQOSqukv3&smid=nytcore-ios-share&referringSource=articleShare)  - The New York Times

---

19. [xcancel.com/trbdrk/statu... x.com/trbdrk/statu...
Midjourney drops video models](https://xcancel.com/trbdrk/status/1933992009955455193)  - xcancel.com

---

20. [John Giannandrea’s complete absence from WWDC may be a clear sign that he won’t be around at Apple for long.](https://www.phonearena.com/news/apple-head-of-ai-not-long-for-cupertino-following-wwdc_id171354)  - PhoneArena![image](download_images/Image20.jpg)



---

21. [ChatGPT's AI coder Codex now lets you generate multiple solutions and cherry-pick the best one](https://www.bleepingcomputer.com/news/artificial-intelligence/chatgpts-ai-coder-codex-now-lets-you-choose-the-best-solution/)  - Bleeping Computer

---

22. [Tomorrow’s financiers are learning to leverage machine learning and AI](https://www.ft.com/content/071dc338-b267-466c-836a-f559609fffd5)  - Financial Times![image](download_images/Image22.jpg)



---

23. [Microsoft Code Researcher: Deep Research Agent for Large Systems Code and Commit History](https://www.microsoft.com/en-us/research/publication/code-researcher-deep-research-agent-for-large-systems-code-and-commit-history/)  - Microsoft

---

24. [ChatGPT Search gets an upgrade as OpenAI takes aim at Google](https://www.bleepingcomputer.com/news/artificial-intelligence/chatgpt-search-gets-an-upgrade-as-openai-takes-aim-at-google/)  - Bleeping Computer

---

25. [Terence Tao says today's AIs generate proofs that look flawless, but the mistakes are subtle.](https://www.reddit.com/r/singularity/comments/1lbzxix/terence_tao_says_todays_ais_pass_the_eye_test_but/)  - Reddit

---

26. [Iranian AI innovator dies in attacks on Tehran](https://www.tehrantimes.com/news/514423/Young-artificial-intelligence-expert-inventor-martyred-in-Israeli)  - Tehran Times![image](download_images/Image26.jpg)



---

27. [Scott Galloway has some choice metaphors to describe AI's impact on workers: 'I think of it as corporate Ozempic'](https://www.businessinsider.com/scott-galloway-ai-impact-on-workers-2025-6)  - Business Insider![image](download_images/Image27.jpg)



---

28. [“I’m really convinced that anybody whose job is done on a computer all day is over. It’s just a matter of time,” one engineer told Michelle Del Rey](https://www.independent.co.uk/news/world/americas/ai-job-layoffs-tech-unemployment-b2769796.html)  - The Independent

---

29. [Nvidia CEO slams Anthropic's chief over his claims of AI taking half of jobs and being unsafe — ‘Don’t do it in a dark room and tell me it’s safe’. Also believes that AI technologies will open more career opportunities in the future.](https://www.tomshardware.com/tech-industry/artificial-intelligence/nvidia-ceo-slams-anthropic-chief-over-claims-of-job-eliminations-says-many-jobs-are-going-to-be-created)  - Tom's Hardware![image](download_images/Image29.jpg)



---

30. [BT boss Kirkby expects AI to deepen job cuts, FT reports](https://www.reuters.com/business/media-telecom/bt-ceo-eyes-deeper-job-cuts-ai-becomes-more-powerful-ft-reports-2025-06-15/)  - Reuters

---

31. [AMA passed a resolution demanding explainable AI tools in medical imaging with third-party verification for safety](https://www.politico.com/newsletters/future-pulse/2025/06/12/american-medical-association-wants-explainable-ai-00402237)  - Politico

---

32. [AI Deepfakes Drove 40% of High-Value Crypto Fraud Last Year: Report](https://decrypt.co/325152/ai-deepfakes-drove-high-value-frauds-report)  - Decrypt![image](download_images/Image32.jpg)



---

33. [AI is defeating biometric security](https://www.cdotrends.com/story/4593/game-over-how-ai-defeating-biometric-security)  - CDOTrends![image](download_images/Image33.jpg)



---

34. [From crypto to transport, AI-powered ID fraud is wrecking industries before they even know it’s coming](https://www.techradar.com/pro/security/synthetic-id-document-fraud-is-exploding-worldwide-thanks-entirely-to-generative-ai-heres-how-to-stay-safe)  - TechRadar![image](download_images/Image34.jpg)



---

35. 
Just a shameless liar. 

He’ll push the line that this was a leftist until the evidence the other way — registered Republican, homeschooling preacher, anti-abortion with a target list of Democrats — becomes too overwhelming, and *then* he’ll argue “we shouldn’t politicize this”



---

Follow the latest AI headlines via [SkynetAndChill.com on Bluesky](https://bsky.app/profile/skynetandchill.com)

In [82]:
stop_index = 999

In [83]:
output_str = ""
for i in sorted(display_dict.keys()):
    if i >= stop_index:
        break
    md_str = f"{i}. "
    md_str += display_dict.get(i)
    if image_dict.get(i):
        md_str += image_dict.get(i)
    md_str += "\n\n---\n\n"
    output_str += md_str

output_str += f"Follow the latest AI headlines via [SkynetAndChill.com on Bluesky](https://bsky.app/profile/skynetandchill.com)"

display(Markdown(output_str))

0. [Economic miracles stem from discovery of deeper insights, not repeating tasks at greater speed. Oxford author of "How Progress Ends: Technology, Innovation, and the Fate of Nations" (sometimes creativity is a search process tho?)](https://www.ft.com/content/55bc5876-254a-4daa-86f8-2cd0d939a866)  - Financial Times![image](download_images/Image0.jpg)



---

1. [Everyone Is Using A.I. for Everything. Is That Bad? So what if we have armies of interns who act like they are high on ketamine?](https://www.nytimes.com/2025/06/16/magazine/using-ai-hard-fork.html)  - The New York Times

---

2. [Chinese start-up applies AI to help Nike, Gap, Urban Revivo predict fashion trends](https://www.scmp.com/tech/tech-trends/article/3314467/how-chinese-ai-start-helps-nike-gap-urban-revivo-predict-fashion-trends)  - South China Morning Post![image](download_images/Image2.jpg)



---

3. [Why the rebuttal to Apple's paper on limits of reasoning models misses the mark.](https://victoramartinez.com/posts/why-claudes-comment-paper-is-a-poor-rebuttal/)  - Victor A Martinez

---

4. [Alexandr Wang waiting for brain implants to have kids: “When we get Neuralink and we get these other technologies, kids who are born with them are gonna learn how to use them in like crazy, crazy ways”](https://economictimes.indiatimes.com/magazines/panache/neuralink-babies-scale-ais-alexandr-wang-says-he-is-waiting-for-elon-musks-brain-chips-before-having-kids/articleshow/121867951.cms?utm_source=contentofinterest&utm_medium=text&utm_campaign=cppst)  - The Economic Times![image](download_images/Image4.jpg)



---

5. [The AI Industry Is Ready to Get Rich off Trump’s Defense Department](https://newrepublic.com/article/196763/ai-industry-trump-defense-department)  - The New Republic![image](download_images/Image5.jpg)



---

6. 




---

7. [AI is the control layer for everything, and race between US and China resembles Cold War.](https://cointelegraph.com/news/ai-race-between-us-china-cold-war-andreessen)  - Cointelegraph

---

8. [Ready or Not, Agentic AI Is Disrupting Corporate Landscapes](https://thenewstack.io/ready-or-not-agentic-ai-is-disrupting-corporate-landscapes/)  - The New Stack![image](download_images/Image8.jpg)



---

9. [AI coding tools upend the 'buy versus build' software equation and threaten the SaaS business model. Sometimes you buy a big SaaS to do 1 or 2 things in your workflow that you could just vibe-code.](https://www.businessinsider.com/ai-coding-tools-buy-versus-build-software-saas-netlify-bolt-2025-6)  - Business Insider![image](download_images/Image9.jpg)



---

10. [the lethal trifecta - 
- access to internal data
- access to external tools
- untrusted prompts / outputs (and any amount of guardrails and sanitizing will eventually be defeated)](https://simonwillison.net/2025/Jun/16/the-lethal-trifecta/)  - Simon Willison’s Weblog![image](download_images/Image10.jpg)



---

11. [Coding agents have crossed a chasm](https://blog.singleton.io/posts/2025-06-14-coding-agents-cross-a-chasm/)  - Singleton Blog

---

12. [Your Brain on ChatGPT: EEG Shows Accumulation of Cognitive Debt when Using an AI Assistant for Essay Writing Task](https://arxiv.org/abs/2506.08872)  - arXiv.org

---

13. 
Elon Musk working on fixing Grok's 'leftist indoctrination', ie lobotomizing it to rant about white genocide.



---

14. [Why Waymo cars became sitting ducks during the L.A. protests.](https://www.nbcnews.com/tech/innovation/waymo-cars-set-fire-sitting-ducks-la-protests-rcna212426)  - NBC News![image](download_images/Image14.jpg)



---

15. [London is a big test for robotaxis. It ain't Phoenix, might not work without people messing with them, cautious taxis gridlocking narrow chaotic streets, needing remote operator intervention and geo-fencing out of certain areas or redesigned roads.](https://www.businessinsider.com/i-took-chaotic-robotaxi-ride-through-london-impressive-one-question-2025-6)  - Business Insider![image](download_images/Image15.jpg)



---

16. 
I was reading this waymo blog post, and remembered that waymo famously drives about 1000x as much in simulation as in the real world. Lines up pretty well with the predictions i made in my "scaling robot learning to Robot GPT" blog post:



---

17. [How AI can help make cities work better for residents. Big opportunities to improve services, and screw up massively.](https://www.technologyreview.com/2025/06/16/1118210/ai-city-government-residents-constituents/)  - MIT Technology Review![image](download_images/Image17.jpg)



---

18. [“If history, per the adage, is written by the winners, then it’s not premature to wonder how the winners of the A.I. race might soon shape the stories that historians tell about the past.”](https://www.nytimes.com/2025/06/16/magazine/ai-history-historians-scholarship.html?unlocked_article_code=1.PU8.EhBm.UkxDQOSqukv3&smid=nytcore-ios-share&referringSource=articleShare)  - The New York Times

---

19. [xcancel.com/trbdrk/statu... x.com/trbdrk/statu...
Midjourney drops video models](https://xcancel.com/trbdrk/status/1933992009955455193)  - xcancel.com

---

20. [John Giannandrea’s complete absence from WWDC may be a clear sign that he won’t be around at Apple for long.](https://www.phonearena.com/news/apple-head-of-ai-not-long-for-cupertino-following-wwdc_id171354)  - PhoneArena![image](download_images/Image20.jpg)



---

21. [ChatGPT's AI coder Codex now lets you generate multiple solutions and cherry-pick the best one](https://www.bleepingcomputer.com/news/artificial-intelligence/chatgpts-ai-coder-codex-now-lets-you-choose-the-best-solution/)  - Bleeping Computer

---

22. [Tomorrow’s financiers are learning to leverage machine learning and AI](https://www.ft.com/content/071dc338-b267-466c-836a-f559609fffd5)  - Financial Times![image](download_images/Image22.jpg)



---

23. [Microsoft Code Researcher: Deep Research Agent for Large Systems Code and Commit History](https://www.microsoft.com/en-us/research/publication/code-researcher-deep-research-agent-for-large-systems-code-and-commit-history/)  - Microsoft

---

24. [ChatGPT Search gets an upgrade as OpenAI takes aim at Google](https://www.bleepingcomputer.com/news/artificial-intelligence/chatgpt-search-gets-an-upgrade-as-openai-takes-aim-at-google/)  - Bleeping Computer

---

25. [Terence Tao says today's AIs generate proofs that look flawless, but the mistakes are subtle.](https://www.reddit.com/r/singularity/comments/1lbzxix/terence_tao_says_todays_ais_pass_the_eye_test_but/)  - Reddit

---

26. [Iranian AI innovator dies in attacks on Tehran](https://www.tehrantimes.com/news/514423/Young-artificial-intelligence-expert-inventor-martyred-in-Israeli)  - Tehran Times![image](download_images/Image26.jpg)



---

27. [Scott Galloway has some choice metaphors to describe AI's impact on workers: 'I think of it as corporate Ozempic'](https://www.businessinsider.com/scott-galloway-ai-impact-on-workers-2025-6)  - Business Insider![image](download_images/Image27.jpg)



---

28. [“I’m really convinced that anybody whose job is done on a computer all day is over. It’s just a matter of time,” one engineer told Michelle Del Rey](https://www.independent.co.uk/news/world/americas/ai-job-layoffs-tech-unemployment-b2769796.html)  - The Independent

---

29. [Nvidia CEO slams Anthropic's chief over his claims of AI taking half of jobs and being unsafe — ‘Don’t do it in a dark room and tell me it’s safe’. Also believes that AI technologies will open more career opportunities in the future.](https://www.tomshardware.com/tech-industry/artificial-intelligence/nvidia-ceo-slams-anthropic-chief-over-claims-of-job-eliminations-says-many-jobs-are-going-to-be-created)  - Tom's Hardware![image](download_images/Image29.jpg)



---

30. [BT boss Kirkby expects AI to deepen job cuts, FT reports](https://www.reuters.com/business/media-telecom/bt-ceo-eyes-deeper-job-cuts-ai-becomes-more-powerful-ft-reports-2025-06-15/)  - Reuters

---

31. [AMA passed a resolution demanding explainable AI tools in medical imaging with third-party verification for safety](https://www.politico.com/newsletters/future-pulse/2025/06/12/american-medical-association-wants-explainable-ai-00402237)  - Politico

---

32. [AI Deepfakes Drove 40% of High-Value Crypto Fraud Last Year: Report](https://decrypt.co/325152/ai-deepfakes-drove-high-value-frauds-report)  - Decrypt![image](download_images/Image32.jpg)



---

33. [AI is defeating biometric security](https://www.cdotrends.com/story/4593/game-over-how-ai-defeating-biometric-security)  - CDOTrends![image](download_images/Image33.jpg)



---

34. [From crypto to transport, AI-powered ID fraud is wrecking industries before they even know it’s coming](https://www.techradar.com/pro/security/synthetic-id-document-fraud-is-exploding-worldwide-thanks-entirely-to-generative-ai-heres-how-to-stay-safe)  - TechRadar![image](download_images/Image34.jpg)



---

35. 
Just a shameless liar. 

He’ll push the line that this was a leftist until the evidence the other way — registered Republican, homeschooling preacher, anti-abortion with a target list of Democrats — becomes too overwhelming, and *then* he’ll argue “we shouldn’t politicize this”



---

Follow the latest AI headlines via [SkynetAndChill.com on Bluesky](https://bsky.app/profile/skynetandchill.com)

In [84]:
import markdown
output_str =""
markdown_extensions = [
    # 'tables',
    # 'fenced_code',
    # 'codehilite',
    'attr_list',
    'def_list',
    # 'footnotes',
    'markdown.extensions.nl2br',
    'markdown.extensions.sane_lists'
]

output_str = ""
output_array = []
for i in sorted(display_dict.keys()):
    md_str=""
    if i >= stop_index:
        break
    if image_dict.get(i):
        md_str += "\n\n"
        md_str += image_dict.get(i)
    md_str += display_dict.get(i)
    md_str += "\n\n---\n\n"
    output_str += md_str
    output_array.append(display_dict.get(i))
    
output_str += f"Follow the latest AI headlines via [SkynetAndChill.com on Bluesky](https://bsky.app/profile/skynetandchill.com)"

output_df = pd.DataFrame(output_array)
output_df.columns = ['markdown']
output_df = output_df.reset_index()
    
display(Markdown(output_str))
html_str = markdown.markdown(output_str, extensions=markdown_extensions)

with open('skynet.html', 'w', encoding="utf-8") as f:
        f.write(html_str)



![image](download_images/Image0.jpg)

[Economic miracles stem from discovery of deeper insights, not repeating tasks at greater speed. Oxford author of "How Progress Ends: Technology, Innovation, and the Fate of Nations" (sometimes creativity is a search process tho?)](https://www.ft.com/content/55bc5876-254a-4daa-86f8-2cd0d939a866)  - Financial Times

---

[Everyone Is Using A.I. for Everything. Is That Bad? So what if we have armies of interns who act like they are high on ketamine?](https://www.nytimes.com/2025/06/16/magazine/using-ai-hard-fork.html)  - The New York Times

---



![image](download_images/Image2.jpg)

[Chinese start-up applies AI to help Nike, Gap, Urban Revivo predict fashion trends](https://www.scmp.com/tech/tech-trends/article/3314467/how-chinese-ai-start-helps-nike-gap-urban-revivo-predict-fashion-trends)  - South China Morning Post

---

[Why the rebuttal to Apple's paper on limits of reasoning models misses the mark.](https://victoramartinez.com/posts/why-claudes-comment-paper-is-a-poor-rebuttal/)  - Victor A Martinez

---



![image](download_images/Image4.jpg)

[Alexandr Wang waiting for brain implants to have kids: “When we get Neuralink and we get these other technologies, kids who are born with them are gonna learn how to use them in like crazy, crazy ways”](https://economictimes.indiatimes.com/magazines/panache/neuralink-babies-scale-ais-alexandr-wang-says-he-is-waiting-for-elon-musks-brain-chips-before-having-kids/articleshow/121867951.cms?utm_source=contentofinterest&utm_medium=text&utm_campaign=cppst)  - The Economic Times

---



![image](download_images/Image5.jpg)

[The AI Industry Is Ready to Get Rich off Trump’s Defense Department](https://newrepublic.com/article/196763/ai-industry-trump-defense-department)  - The New Republic

---






---

[AI is the control layer for everything, and race between US and China resembles Cold War.](https://cointelegraph.com/news/ai-race-between-us-china-cold-war-andreessen)  - Cointelegraph

---



![image](download_images/Image8.jpg)

[Ready or Not, Agentic AI Is Disrupting Corporate Landscapes](https://thenewstack.io/ready-or-not-agentic-ai-is-disrupting-corporate-landscapes/)  - The New Stack

---



![image](download_images/Image9.jpg)

[AI coding tools upend the 'buy versus build' software equation and threaten the SaaS business model. Sometimes you buy a big SaaS to do 1 or 2 things in your workflow that you could just vibe-code.](https://www.businessinsider.com/ai-coding-tools-buy-versus-build-software-saas-netlify-bolt-2025-6)  - Business Insider

---



![image](download_images/Image10.jpg)

[the lethal trifecta - 
- access to internal data
- access to external tools
- untrusted prompts / outputs (and any amount of guardrails and sanitizing will eventually be defeated)](https://simonwillison.net/2025/Jun/16/the-lethal-trifecta/)  - Simon Willison’s Weblog

---

[Coding agents have crossed a chasm](https://blog.singleton.io/posts/2025-06-14-coding-agents-cross-a-chasm/)  - Singleton Blog

---

[Your Brain on ChatGPT: EEG Shows Accumulation of Cognitive Debt when Using an AI Assistant for Essay Writing Task](https://arxiv.org/abs/2506.08872)  - arXiv.org

---


Elon Musk working on fixing Grok's 'leftist indoctrination', ie lobotomizing it to rant about white genocide.



---



![image](download_images/Image14.jpg)

[Why Waymo cars became sitting ducks during the L.A. protests.](https://www.nbcnews.com/tech/innovation/waymo-cars-set-fire-sitting-ducks-la-protests-rcna212426)  - NBC News

---



![image](download_images/Image15.jpg)

[London is a big test for robotaxis. It ain't Phoenix, might not work without people messing with them, cautious taxis gridlocking narrow chaotic streets, needing remote operator intervention and geo-fencing out of certain areas or redesigned roads.](https://www.businessinsider.com/i-took-chaotic-robotaxi-ride-through-london-impressive-one-question-2025-6)  - Business Insider

---


I was reading this waymo blog post, and remembered that waymo famously drives about 1000x as much in simulation as in the real world. Lines up pretty well with the predictions i made in my "scaling robot learning to Robot GPT" blog post:



---



![image](download_images/Image17.jpg)

[How AI can help make cities work better for residents. Big opportunities to improve services, and screw up massively.](https://www.technologyreview.com/2025/06/16/1118210/ai-city-government-residents-constituents/)  - MIT Technology Review

---

[“If history, per the adage, is written by the winners, then it’s not premature to wonder how the winners of the A.I. race might soon shape the stories that historians tell about the past.”](https://www.nytimes.com/2025/06/16/magazine/ai-history-historians-scholarship.html?unlocked_article_code=1.PU8.EhBm.UkxDQOSqukv3&smid=nytcore-ios-share&referringSource=articleShare)  - The New York Times

---

[xcancel.com/trbdrk/statu... x.com/trbdrk/statu...
Midjourney drops video models](https://xcancel.com/trbdrk/status/1933992009955455193)  - xcancel.com

---



![image](download_images/Image20.jpg)

[John Giannandrea’s complete absence from WWDC may be a clear sign that he won’t be around at Apple for long.](https://www.phonearena.com/news/apple-head-of-ai-not-long-for-cupertino-following-wwdc_id171354)  - PhoneArena

---

[ChatGPT's AI coder Codex now lets you generate multiple solutions and cherry-pick the best one](https://www.bleepingcomputer.com/news/artificial-intelligence/chatgpts-ai-coder-codex-now-lets-you-choose-the-best-solution/)  - Bleeping Computer

---



![image](download_images/Image22.jpg)

[Tomorrow’s financiers are learning to leverage machine learning and AI](https://www.ft.com/content/071dc338-b267-466c-836a-f559609fffd5)  - Financial Times

---

[Microsoft Code Researcher: Deep Research Agent for Large Systems Code and Commit History](https://www.microsoft.com/en-us/research/publication/code-researcher-deep-research-agent-for-large-systems-code-and-commit-history/)  - Microsoft

---

[ChatGPT Search gets an upgrade as OpenAI takes aim at Google](https://www.bleepingcomputer.com/news/artificial-intelligence/chatgpt-search-gets-an-upgrade-as-openai-takes-aim-at-google/)  - Bleeping Computer

---

[Terence Tao says today's AIs generate proofs that look flawless, but the mistakes are subtle.](https://www.reddit.com/r/singularity/comments/1lbzxix/terence_tao_says_todays_ais_pass_the_eye_test_but/)  - Reddit

---



![image](download_images/Image26.jpg)

[Iranian AI innovator dies in attacks on Tehran](https://www.tehrantimes.com/news/514423/Young-artificial-intelligence-expert-inventor-martyred-in-Israeli)  - Tehran Times

---



![image](download_images/Image27.jpg)

[Scott Galloway has some choice metaphors to describe AI's impact on workers: 'I think of it as corporate Ozempic'](https://www.businessinsider.com/scott-galloway-ai-impact-on-workers-2025-6)  - Business Insider

---

[“I’m really convinced that anybody whose job is done on a computer all day is over. It’s just a matter of time,” one engineer told Michelle Del Rey](https://www.independent.co.uk/news/world/americas/ai-job-layoffs-tech-unemployment-b2769796.html)  - The Independent

---



![image](download_images/Image29.jpg)

[Nvidia CEO slams Anthropic's chief over his claims of AI taking half of jobs and being unsafe — ‘Don’t do it in a dark room and tell me it’s safe’. Also believes that AI technologies will open more career opportunities in the future.](https://www.tomshardware.com/tech-industry/artificial-intelligence/nvidia-ceo-slams-anthropic-chief-over-claims-of-job-eliminations-says-many-jobs-are-going-to-be-created)  - Tom's Hardware

---

[BT boss Kirkby expects AI to deepen job cuts, FT reports](https://www.reuters.com/business/media-telecom/bt-ceo-eyes-deeper-job-cuts-ai-becomes-more-powerful-ft-reports-2025-06-15/)  - Reuters

---

[AMA passed a resolution demanding explainable AI tools in medical imaging with third-party verification for safety](https://www.politico.com/newsletters/future-pulse/2025/06/12/american-medical-association-wants-explainable-ai-00402237)  - Politico

---



![image](download_images/Image32.jpg)

[AI Deepfakes Drove 40% of High-Value Crypto Fraud Last Year: Report](https://decrypt.co/325152/ai-deepfakes-drove-high-value-frauds-report)  - Decrypt

---



![image](download_images/Image33.jpg)

[AI is defeating biometric security](https://www.cdotrends.com/story/4593/game-over-how-ai-defeating-biometric-security)  - CDOTrends

---



![image](download_images/Image34.jpg)

[From crypto to transport, AI-powered ID fraud is wrecking industries before they even know it’s coming](https://www.techradar.com/pro/security/synthetic-id-document-fraud-is-exploding-worldwide-thanks-entirely-to-generative-ai-heres-how-to-stay-safe)  - TechRadar

---


Just a shameless liar. 

He’ll push the line that this was a leftist until the evidence the other way — registered Republican, homeschooling preacher, anti-abortion with a target list of Democrats — becomes too overwhelming, and *then* he’ll argue “we shouldn’t politicize this”



---

Follow the latest AI headlines via [SkynetAndChill.com on Bluesky](https://bsky.app/profile/skynetandchill.com)

10281

In [85]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.output_parsers import PydanticOutputParser
from pydantic import RootModel
import os

from pydantic import BaseModel, RootModel, Field
from typing import List

class ItemList(BaseModel):
    """List of story indexes"""
    items: List[int] = Field(description="The ordered list of story indexes")


system_prompt = """
# ROLE
You are a newsletter editor.

# TASK
Group and reorder the items in the Markdown newsletter supplied below so that:
1. Items that cover similar companies, individuals, technologies or topics are **next to each other** (i.e., logically grouped).
2. Groups appear from **most important** to **least important** according to the **IMPORTANCE FACTORS** listed below.
3. Importance of groups is determined by the most imporant story 

# HOW TO GROUP
- If two or more items discuss the same organization, person, technology, or topic, keep them all together.
- Within any group, order by importance using **IMPORTANCE FACTORS**.

# HOW TO ORDER
- Lead with stories that have highest importance per the **IMPORTANCE FACTORS** below
- Follow with narrower‑scope or opinion pieces.
- End with lighter or human‑interest items.

# IMPORTANCE FACTORS
1 Magnitude of impact : large user base, $ at stake, broad social reach
2 Novelty : breaks conceptual ground, not a minor iteration
3 Authority : reputable institution, peer-review, regulatory filing, on-record executive
4 Verifiability : code, data, benchmarks, or other concrete evidence provided
5 Timeliness : early signal of an important trend or shift
6 Breadth : cross-industry / cross-disciplinary / international implications
7 Strategic consequence : shifts competitive, power, or policy dynamics
8 Financial materiality : clear valuation or growth or revenue impact
9 Risk & safety : raises or mitigates critical alignment, security, or ethical risk
10 Actionability : informs concrete decisions for investors, policymakers, practitioners
11 Longevity : likely to matter or be referred to in coming days, weeks, or months
12 Independent corroboration : confirmed by multiple sources or datasets
13 Clarity : enough technical/context detail to judge merit; minimal hype

# INPUT FORMAT
You will receive a JSON array of items, with an index and markdown summary of the item.

# OUTPUT FORMAT
- Return **only** the reordered indexes using the provided schema.
- Do **not** alter the wording of individual items (other than correcting spelling and punctuation, and normalizing spaces or line breaks as needed).
- Insert one blank line between each logical group for readability.
- Do not output anything other than the newsletter itself—no headers, explanations, or commentary.

"""

user_prompt = """
{input_text}
"""

prompt_template = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("user", user_prompt)
])

model = ChatOpenAI(
        model="gpt-4.1",  
        openai_api_key=os.getenv("OPENAI_API_KEY")
    )
# Create the chain
chain = prompt_template | model.with_structured_output(ItemList)
response = chain.invoke({"input_text": output_df.to_json(orient="records")})


In [86]:
len(response.items)


36

In [87]:
output_str = ""
for i in response.items:
    md_str = ""
#     output_str += f"{i}\n\n"
    if image_dict.get(i):
        md_str += "\n\n"
        md_str += image_dict.get(i)
    md_str += display_dict.get(i)
    md_str += "\n\n---\n\n"
    output_str += md_str
    
output_str += f"Follow the latest AI headlines via [SkynetAndChill.com on Bluesky](https://bsky.app/profile/skynetandchill.com)"

display(Markdown(output_str))
html_str = markdown.markdown(output_str, extensions=markdown_extensions)

with open('skynet.html', 'w', encoding="utf-8") as f:
    f.write(html_str)
    

[ChatGPT Search gets an upgrade as OpenAI takes aim at Google](https://www.bleepingcomputer.com/news/artificial-intelligence/chatgpt-search-gets-an-upgrade-as-openai-takes-aim-at-google/)  - Bleeping Computer

---

[ChatGPT's AI coder Codex now lets you generate multiple solutions and cherry-pick the best one](https://www.bleepingcomputer.com/news/artificial-intelligence/chatgpts-ai-coder-codex-now-lets-you-choose-the-best-solution/)  - Bleeping Computer

---

[Microsoft Code Researcher: Deep Research Agent for Large Systems Code and Commit History](https://www.microsoft.com/en-us/research/publication/code-researcher-deep-research-agent-for-large-systems-code-and-commit-history/)  - Microsoft

---

[Coding agents have crossed a chasm](https://blog.singleton.io/posts/2025-06-14-coding-agents-cross-a-chasm/)  - Singleton Blog

---



![image](download_images/Image9.jpg)

[AI coding tools upend the 'buy versus build' software equation and threaten the SaaS business model. Sometimes you buy a big SaaS to do 1 or 2 things in your workflow that you could just vibe-code.](https://www.businessinsider.com/ai-coding-tools-buy-versus-build-software-saas-netlify-bolt-2025-6)  - Business Insider

---



![image](download_images/Image8.jpg)

[Ready or Not, Agentic AI Is Disrupting Corporate Landscapes](https://thenewstack.io/ready-or-not-agentic-ai-is-disrupting-corporate-landscapes/)  - The New Stack

---



![image](download_images/Image29.jpg)

[Nvidia CEO slams Anthropic's chief over his claims of AI taking half of jobs and being unsafe — ‘Don’t do it in a dark room and tell me it’s safe’. Also believes that AI technologies will open more career opportunities in the future.](https://www.tomshardware.com/tech-industry/artificial-intelligence/nvidia-ceo-slams-anthropic-chief-over-claims-of-job-eliminations-says-many-jobs-are-going-to-be-created)  - Tom's Hardware

---

[BT boss Kirkby expects AI to deepen job cuts, FT reports](https://www.reuters.com/business/media-telecom/bt-ceo-eyes-deeper-job-cuts-ai-becomes-more-powerful-ft-reports-2025-06-15/)  - Reuters

---

[“I’m really convinced that anybody whose job is done on a computer all day is over. It’s just a matter of time,” one engineer told Michelle Del Rey](https://www.independent.co.uk/news/world/americas/ai-job-layoffs-tech-unemployment-b2769796.html)  - The Independent

---



![image](download_images/Image27.jpg)

[Scott Galloway has some choice metaphors to describe AI's impact on workers: 'I think of it as corporate Ozempic'](https://www.businessinsider.com/scott-galloway-ai-impact-on-workers-2025-6)  - Business Insider

---

[Everyone Is Using A.I. for Everything. Is That Bad? So what if we have armies of interns who act like they are high on ketamine?](https://www.nytimes.com/2025/06/16/magazine/using-ai-hard-fork.html)  - The New York Times

---

[“If history, per the adage, is written by the winners, then it’s not premature to wonder how the winners of the A.I. race might soon shape the stories that historians tell about the past.”](https://www.nytimes.com/2025/06/16/magazine/ai-history-historians-scholarship.html?unlocked_article_code=1.PU8.EhBm.UkxDQOSqukv3&smid=nytcore-ios-share&referringSource=articleShare)  - The New York Times

---



![image](download_images/Image22.jpg)

[Tomorrow’s financiers are learning to leverage machine learning and AI](https://www.ft.com/content/071dc338-b267-466c-836a-f559609fffd5)  - Financial Times

---



![image](download_images/Image17.jpg)

[How AI can help make cities work better for residents. Big opportunities to improve services, and screw up massively.](https://www.technologyreview.com/2025/06/16/1118210/ai-city-government-residents-constituents/)  - MIT Technology Review

---



![image](download_images/Image2.jpg)

[Chinese start-up applies AI to help Nike, Gap, Urban Revivo predict fashion trends](https://www.scmp.com/tech/tech-trends/article/3314467/how-chinese-ai-start-helps-nike-gap-urban-revivo-predict-fashion-trends)  - South China Morning Post

---

[AI is the control layer for everything, and race between US and China resembles Cold War.](https://cointelegraph.com/news/ai-race-between-us-china-cold-war-andreessen)  - Cointelegraph

---



![image](download_images/Image5.jpg)

[The AI Industry Is Ready to Get Rich off Trump’s Defense Department](https://newrepublic.com/article/196763/ai-industry-trump-defense-department)  - The New Republic

---



![image](download_images/Image4.jpg)

[Alexandr Wang waiting for brain implants to have kids: “When we get Neuralink and we get these other technologies, kids who are born with them are gonna learn how to use them in like crazy, crazy ways”](https://economictimes.indiatimes.com/magazines/panache/neuralink-babies-scale-ais-alexandr-wang-says-he-is-waiting-for-elon-musks-brain-chips-before-having-kids/articleshow/121867951.cms?utm_source=contentofinterest&utm_medium=text&utm_campaign=cppst)  - The Economic Times

---



![image](download_images/Image20.jpg)

[John Giannandrea’s complete absence from WWDC may be a clear sign that he won’t be around at Apple for long.](https://www.phonearena.com/news/apple-head-of-ai-not-long-for-cupertino-following-wwdc_id171354)  - PhoneArena

---

[Why the rebuttal to Apple's paper on limits of reasoning models misses the mark.](https://victoramartinez.com/posts/why-claudes-comment-paper-is-a-poor-rebuttal/)  - Victor A Martinez

---

[Terence Tao says today's AIs generate proofs that look flawless, but the mistakes are subtle.](https://www.reddit.com/r/singularity/comments/1lbzxix/terence_tao_says_todays_ais_pass_the_eye_test_but/)  - Reddit

---

[Your Brain on ChatGPT: EEG Shows Accumulation of Cognitive Debt when Using an AI Assistant for Essay Writing Task](https://arxiv.org/abs/2506.08872)  - arXiv.org

---

[AMA passed a resolution demanding explainable AI tools in medical imaging with third-party verification for safety](https://www.politico.com/newsletters/future-pulse/2025/06/12/american-medical-association-wants-explainable-ai-00402237)  - Politico

---



![image](download_images/Image32.jpg)

[AI Deepfakes Drove 40% of High-Value Crypto Fraud Last Year: Report](https://decrypt.co/325152/ai-deepfakes-drove-high-value-frauds-report)  - Decrypt

---



![image](download_images/Image34.jpg)

[From crypto to transport, AI-powered ID fraud is wrecking industries before they even know it’s coming](https://www.techradar.com/pro/security/synthetic-id-document-fraud-is-exploding-worldwide-thanks-entirely-to-generative-ai-heres-how-to-stay-safe)  - TechRadar

---



![image](download_images/Image33.jpg)

[AI is defeating biometric security](https://www.cdotrends.com/story/4593/game-over-how-ai-defeating-biometric-security)  - CDOTrends

---



![image](download_images/Image10.jpg)

[the lethal trifecta - 
- access to internal data
- access to external tools
- untrusted prompts / outputs (and any amount of guardrails and sanitizing will eventually be defeated)](https://simonwillison.net/2025/Jun/16/the-lethal-trifecta/)  - Simon Willison’s Weblog

---

[xcancel.com/trbdrk/statu... x.com/trbdrk/statu...
Midjourney drops video models](https://xcancel.com/trbdrk/status/1933992009955455193)  - xcancel.com

---



![image](download_images/Image26.jpg)

[Iranian AI innovator dies in attacks on Tehran](https://www.tehrantimes.com/news/514423/Young-artificial-intelligence-expert-inventor-martyred-in-Israeli)  - Tehran Times

---



![image](download_images/Image14.jpg)

[Why Waymo cars became sitting ducks during the L.A. protests.](https://www.nbcnews.com/tech/innovation/waymo-cars-set-fire-sitting-ducks-la-protests-rcna212426)  - NBC News

---



![image](download_images/Image15.jpg)

[London is a big test for robotaxis. It ain't Phoenix, might not work without people messing with them, cautious taxis gridlocking narrow chaotic streets, needing remote operator intervention and geo-fencing out of certain areas or redesigned roads.](https://www.businessinsider.com/i-took-chaotic-robotaxi-ride-through-london-impressive-one-question-2025-6)  - Business Insider

---


I was reading this waymo blog post, and remembered that waymo famously drives about 1000x as much in simulation as in the real world. Lines up pretty well with the predictions i made in my "scaling robot learning to Robot GPT" blog post:



---



![image](download_images/Image0.jpg)

[Economic miracles stem from discovery of deeper insights, not repeating tasks at greater speed. Oxford author of "How Progress Ends: Technology, Innovation, and the Fate of Nations" (sometimes creativity is a search process tho?)](https://www.ft.com/content/55bc5876-254a-4daa-86f8-2cd0d939a866)  - Financial Times

---


Elon Musk working on fixing Grok's 'leftist indoctrination', ie lobotomizing it to rant about white genocide.



---


Just a shameless liar. 

He’ll push the line that this was a leftist until the evidence the other way — registered Republican, homeschooling preacher, anti-abortion with a target list of Democrats — becomes too overwhelming, and *then* he’ll argue “we shouldn’t politicize this”



---






---

Follow the latest AI headlines via [SkynetAndChill.com on Bluesky](https://bsky.app/profile/skynetandchill.com)

10281

In [88]:
pyperclip.copy( output_str )


In [ ]:
date_str = data.feed[0].post.record.created_at
dt = datetime.fromisoformat(date_str.rstrip('Z'))
dt

In [ ]:

# Get the current datetime
now = datetime.now()

# Create a timedelta of 30 hours
delta = timedelta(hours=30)

# Subtract 30 hours from the current datetime
thirty_hours_ago = now - delta

print(thirty_hours_ago)


In [ ]:
from typing import TypedDict, Annotated

class Point2D(TypedDict, total=False):
    x: int
    y: int
    label: str

a: Point2D = {'x': 1, 'y': 2, }  # 


In [ ]:
a

In [ ]:
import numpy as np

def decay_exponential(t, N0):
    k = np.log(2)  
    return N0 * np.exp(-k * t)

2*decay_exponential(4,1)-1

In [ ]:
```mermaid
flowchart LR

A[Hard] -->|Text| B(Round)
B --> C{Decision}
C -->|One| D[Result 1]
C -->|Two| E[Result 2]


In [ ]:
from datetime import datetime
from zoneinfo import ZoneInfo

dt_str = '2025-05-24 16:00'
naive_dt = datetime.strptime(dt_str, '%Y-%m-%d %H:%M')

# CORRECT: Attach US Eastern timezone *with DST handled*
eastern_dt = naive_dt.replace(tzinfo=ZoneInfo("America/New_York"))

# Convert to UTC
utc_dt = eastern_dt.astimezone(ZoneInfo("UTC"))
print(utc_dt)

In [ ]:
from chromadb.config import Settings
from chromadb.utils import embedding_functions

import chromadb

# Set up the client
client = chromadb.PersistentClient(
    path="chromadb/chroma_articles/", settings=Settings(allow_reset=True)
)


# Get the collection
openai_ef = embedding_functions.OpenAIEmbeddingFunction(
    model_name="text-embedding-3-large",
    api_key=os.getenv("OPENAI_API_KEY")
)

collection = client.get_collection("articles", embedding_function=openai_ef)

# Count the documents
num_docs = collection.count()
print(f"Number of documents in 'articles': {num_docs}")

In [ ]:
from datetime import datetime, timedelta, timezone
from zoneinfo import ZoneInfo

before_date = '2025-05-25 16:30'
if before_date:
    naive_dt = datetime.strptime(before_date, '%Y-%m-%d %H:%M')
    eastern_dt = naive_dt.replace(
        tzinfo=ZoneInfo("America/New_York"))
    utc_dt = eastern_dt.astimezone(ZoneInfo("UTC"))
    before_date = utc_dt.strftime('%Y-%m-%d %H:%M')


In [ ]:
doc = collection.get(limit=1)
doc_str = doc['documents'][0]
doc_str2 = "\n".join(doc_str.split("\n")[12:])

In [ ]:
results = collection.query(
    query_texts=[doc_str2],
    n_results=1,  # number of similar documents to return
    include=["distances", "metadatas"],
#     where={'created': {'$lt': 2025-05-25 20:30'}
)

In [ ]:
results


In [ ]:
before_date


In [ ]:
                where_filter = {"created": f"$lt: {before_date}"}


In [ ]:
where_filter

In [ ]:
from typing import List 
import numpy as np 

def mmr(
    ratings: List[float],
    doc_embeddings: List[np.ndarray],
    lambda_param: float = 0.5,
    top_k: int = 50
) -> List[int]:
    """MMR (Maximal Marginal Relevance) algorithm for selecting a diverse set of documents.
    Combines normalized score and novelty vs previous items

    Args:
        doc_embeddings (List[np.ndarray]): embeddings of documents
        ratings (List[float]): ratings of documents, should be positive, we will scale to max 1
        lambda_param (float, optional): balance between rating and novelty. Defaults to 0.5.
        top_k (int, optional): number of documents to select. Defaults to 50.

    Returns:
        List[int]: indices of selected documents
    """
    selected = []
    candidates = list(range(len(doc_embeddings)))

    ratings = [rating / max(ratings) for rating in ratings]

    while len(selected) < top_k and candidates:
        if not selected:  # none yet
            # start with highest scoring document
            idx = np.argmax(ratings)
            selected.append(idx)  # add to selected
            candidates.remove(idx)  # remove from candidates
            continue

        scores = []
        for idx in candidates:
            rating = ratings[idx]
            # find most similar selected document to this doc idx
            # first array has 1 element so we get a 1x array
            max_sim_to_selected = max(cosine_similarity(
                [doc_embeddings[idx]],  # array with one doc
                [doc_embeddings[i]
                    for i in selected]  # array with all selected docs
            )[0])
            mmr_score = lambda_param * rating - \
                (1 - lambda_param) * max_sim_to_selected
            scores.append((mmr_score, idx))

        # Select candidate with highest MMR score
        # scores is a list of (score, idx), max does lexicographic sort so we get the highest score
        _, best_idx = max(scores)
        selected.append(best_idx)
        candidates.remove(best_idx)

    return selected